In [ ]:
import numpy as np
import brainlit
import scipy
from brainlit.utils import swc
from cloudvolume.exceptions import SkeletonDecodeError
from brainlit.algorithms.trace_analysis.fit_spline import GeometricGraph
from brainlit.algorithms.trace_analysis.spline_fxns import curvature, torsion
import os
from pathlib import Path
import pandas as pd
from networkx.readwrite import json_graph
import json
import matplotlib.pyplot as plt
from statsmodels.graphics import tsaplots

In [ ]:
def auto_corr(series, length):
    series = series - np.mean(series)
    correlation = np.correlate(series, series, mode="full")
    
    middle_idx = int((len(correlation)-1)/2)
    correlation = correlation[middle_idx:]
    correlation = correlation/np.dot(series,series)
    
    l = len(correlation)
    
    if l > length:
        correlation = correlation[:length]
    if l < length:
        correlation = np.concatenate([correlation, np.zeros((length-l))])
    
    return correlation

# Load Data from Brain 1

In [ ]:
#specify brain1 or brain2 below
brain = "brain1"
#specify length definition below (0-length from root, 1-relative length from root, 2-relative length along segment)
length_def = 1

root_dir = Path(os.path.abspath('')).parents[1]
experiment_dir = os.path.join(root_dir, "axon_geometry")
data_dir = os.path.join(experiment_dir, "data", brain)
segments_swc_dir = os.path.join(data_dir, "segments_swc")
trace_data_dir = os.path.join(data_dir, "trace_data")
trace_data_dir = os.path.join(trace_data_dir, "1")
print(f"Directory where swcs reside: {segments_swc_dir}")

In [ ]:
max_id = 300
seg_lengths = []
mean_torsions = []
mean_curvatures = []
d_from_root = []
torsions = []
curvatures = []
total=0

corr_length=25

corr_length=corr_length+1 #to be right end inclusive
running_correlation_curv = np.zeros((corr_length))
total_segments_curv = 0
running_correlation_tor = np.zeros((corr_length))
total_segments_tor = 0

for i in np.arange(0, max_id):
    i = int(i)
    trace_data_path = os.path.join(trace_data_dir, "{}.npy".format(i))
    if os.path.exists(trace_data_path) is True:
        trace_data = np.load(trace_data_path, allow_pickle=True)
        print("Loaded segment {}".format(i))
        total = total+1

        for node in trace_data:
            # print(node)
            seg_lengths.append(node["seg_length"])
            mean_curvatures.append(node["mean_curvature"])
            mean_torsions.append(node["mean_torsion"])
            _curvatures = node["curvature"]
            _torsions = node["torsion"]
            _torsions = np.abs(_torsions)
            assert len(_curvatures) == len(_torsions)
            
            if np.var(_curvatures) > 0:
                correlation_curv = auto_corr(_curvatures, corr_length)
                running_correlation_curv = running_correlation_curv + correlation_curv
                total_segments_curv = total_segments_curv + 1
            
            if np.var(_torsions) > 0:
                correlation_tor = auto_corr(_torsions, corr_length)
                running_correlation_tor = running_correlation_tor + correlation_tor
                total_segments_tor = total_segments_tor + 1
                
correlation_curv_1 = running_correlation_curv / total_segments_curv
correlation_tor_1 = running_correlation_tor / total_segments_tor
print(f"Curvature autocorrelation at lag=14 microns: {correlation_curv_1[14]}")
print(f"Torsion autocorrelation at lag=14 microns: {correlation_tor_1[14]}")

# Load Data from Brain 2

In [ ]:
#specify brain1 or brain2 below
brain = "brain2"
#specify length definition below (0-length from root, 1-relative length from root, 2-relative length along segment)
length_def = 1

root_dir = Path(os.path.abspath('')).parents[1]
experiment_dir = os.path.join(root_dir, "axon_geometry")
data_dir = os.path.join(experiment_dir, "data", brain)
segments_swc_dir = os.path.join(data_dir, "segments_swc")
trace_data_dir = os.path.join(data_dir, "trace_data")
trace_data_dir = os.path.join(trace_data_dir, "1")
print(f"Directory where swcs reside: {segments_swc_dir}")

In [ ]:
max_id = 300
seg_lengths = []
mean_torsions = []
mean_curvatures = []
d_from_root = []
torsions = []
curvatures = []
total=0

corr_length=25

corr_length=corr_length+1 #to be right end inclusive
running_correlation_curv = np.zeros((corr_length))
total_segments_curv = 0
running_correlation_tor = np.zeros((corr_length))
total_segments_tor = 0

for i in np.arange(0, max_id):
    i = int(i)
    trace_data_path = os.path.join(trace_data_dir, "{}.npy".format(i))
    if os.path.exists(trace_data_path) is True:
        trace_data = np.load(trace_data_path, allow_pickle=True)
        print("Loaded segment {}".format(i))
        total = total+1

        for node in trace_data:
            # print(node)
            seg_lengths.append(node["seg_length"])
            mean_curvatures.append(node["mean_curvature"])
            mean_torsions.append(node["mean_torsion"])
            _curvatures = node["curvature"]
            _torsions = node["torsion"]
            _torsions = np.abs(_torsions)
            assert len(_curvatures) == len(_torsions)
            
            if np.var(_curvatures) > 0:
                correlation_curv = auto_corr(_curvatures, corr_length)
                running_correlation_curv = running_correlation_curv + correlation_curv
                total_segments_curv = total_segments_curv + 1
            
            if np.var(_torsions) > 0:
                correlation_tor = auto_corr(_torsions, corr_length)
                running_correlation_tor = running_correlation_tor + correlation_tor
                total_segments_tor = total_segments_tor + 1
                
correlation_curv_2 = running_correlation_curv / total_segments_curv
correlation_tor_2 = running_correlation_tor / total_segments_tor
print(f"Curvature autocorrelation at lag=14 microns: {correlation_curv_2[14]}")
print(f"Torsion autocorrelation at lag=14 microns: {correlation_tor_2[14]}")

In [ ]:
fig = plt.figure(figsize=(20, 8))
axes = fig.subplots(1, 2)
GRAY = "#999999"
TITLE_TYPE_SETTINGS = {"fontname": "Arial", "size": 20}
SUP_TITLE_TYPE_SETTINGS = {"fontname": "Arial", "size": 24}
plt.rc("font", family="Arial", size=20)

ax = axes[0]
ax.spines["bottom"].set_color(GRAY)
ax.spines["top"].set_color(GRAY)
ax.spines["right"].set_color(GRAY)
ax.spines["left"].set_color(GRAY)
ax.tick_params(axis="both", colors=GRAY, labelsize="large")


ax.plot(
    np.arange(corr_length),
    correlation_curv_1,
    color="#377eb8",
    lw=2,
    label="Curvature"
)
ax.plot(
    np.arange(corr_length),
    correlation_tor_1,
    color="#e41a1c",
    lw=2,
    label="Torsion"
)
ax.set_title("Brain 1")
ax.set_xlabel(r"Lag ($\mu m$)", fontsize=22)
ax.set_ylabel(r"Autocorrelation", fontsize=22)
ax.set_ylim([-0.5, 1])
ax.grid()
leg = ax.legend(loc=4)
leg.get_frame().set_edgecolor(GRAY)

leg = ax.legend(loc=4)
leg.get_frame().set_edgecolor(GRAY)

ax = axes[1]
ax.spines["bottom"].set_color(GRAY)
ax.spines["top"].set_color(GRAY)
ax.spines["right"].set_color(GRAY)
ax.spines["left"].set_color(GRAY)
ax.tick_params(axis="both", colors=GRAY, labelsize="large")


ax.plot(
    np.arange(corr_length),
    correlation_curv_2,
    color="#377eb8",
    lw=2,
    label="Curvature"
)
ax.plot(
    np.arange(corr_length),
    correlation_tor_2,
    color="#e41a1c",
    lw=2,
    label="Torsion"
)
ax.set_title("Brain 2")
ax.set_xlabel(r"Lag ($\mu m$)", fontsize=22)
ax.set_ylabel(r"Autocorrelation", fontsize=22)
ax.set_ylim([-0.5, 1])
ax.grid()
leg = ax.legend(loc=4)
leg.get_frame().set_edgecolor(GRAY)

leg = ax.legend(loc=4)
leg.get_frame().set_edgecolor(GRAY)

fig.suptitle("Autocorrelation of Frenet-Serret Parameters")

plt.savefig(os.path.join(experiment_dir, "figures/autocorrelation.eps"))
plt.savefig(os.path.join(experiment_dir, "figures/autocorrelation.jpg"))